DISTRIBUTION STATEMENT A. Approved for public release: distribution is unlimited.

CG-LS.ipynb
* Created by: Carter Lyons

Contains:
* Compound Gaussian Least Squares (CG-LS) iterative algorithm. Estimates signal representation coefficients from linear measurements.
* For more information reference:
    1. Lyons C., Raj R. G., and Cheney M. (2023). "A Compound Gaussian Network for Solving Linear Inverse Problems," In-Review.
    2. Lyons C., Raj R. G., and Cheney M. (2022). "CG-Net: A Compound Gaussian Prior Based Unrolled Imaging Network," in *2022 IEEE Asia-Pacific Signal and Information Processing Association Annual Summit and Conference*, pp. 623-629.
    
Package Requirements (those required in Initialize.py):
* pandas, matplotlib

In [1]:
from skimage.metrics import structural_similarity as ssim  
import numpy as np
import time
import copy
import matplotlib.pyplot as plt
import pandas as pd
import Initialize

img_size = 32  # Height and width of input images 
B = Initialize.create_measurements('barbara.png', [[154, 154+img_size],[64,64+img_size]], import_phi = (False, [], 'bior1.1'), import_psi = (False, [], 15))  # From Initialize.py: Create Radon transform matrix. Create biorthogonal wavelet matrix. Use these matrices to form measurements of input 'Barbara' image

# Alternating $u$ and $z$ estimation using Steepest Descent

We alternate between the following minimzations
\begin{align}
&\min_{u} ||y - A(z\odot u) ||_2^2 + \lambda ||u||_2^2 \\
&\min_{z} ||y - A(z\odot u) ||_2^2 + \lambda ||f(z)||_2^2
\end{align}

The step size $t$ in this method is calculated by an Armijo's line search: See
https://en.wikipedia.org/wiki/Backtracking_line_search for a simple description.

In [3]:
# Class to set up and perform steepest descent estimation on z
class Zest:
    
    # Class initilization function. When class Zest is called the inputs for __init__ can be passed to it to create all of the variables below and initalize the class.
    def __init__(self, MaxSteps, lamb, mu, Psi, Phi, y, method = 'g', tol = 10**(-5), epsilon = 10**(-5), threshold = (1, np.e), dom = (0,np.inf), ArmijoTau = 0.5, ArmijoBeta = 0.5):
        self.N = MaxSteps      # Number of steepest descent steps
        self.lamb = lamb           # Regularization parameter
        self.mu = mu               # Regularization parameter
        self.A = Psi@Phi          # Measurement matrix 
        self.y = y             # Measurements
        self.method = method # 'g' for gradient descent and 'n' for Newton Descent
        self.tol = tol             # Tolerance for terminating. If the change in norm between steps < tol then estimation ends
        self.eps = epsilon  # Stabilizing parameter for method = 'n'
        self.threshold = threshold # Threshold parameters. If z_i <= 0 then z_i = threshold[0] and z_i > e then z_i = treshold[1]. Default is (e^-3, e)
        self.dom = dom  # Domain of optimization
        self.tau = ArmijoTau          # Armijo backtracking line search parameter. c should be chosen in (0, 1) and is a control parameter           
        self.beta = ArmijoBeta      # Armijo backtracking line search parameter. tau should be chosen in (0, 1). Shrink the step size by factor of tau each backtrack iteration
       
        self.z = self.Projection(self.A.T@y)   # Initial estimate of z.     
        self.u = np.ones(1)        # Starting placeholder for u
      
    def f(self, Z):     # Inverse nonlinearity between z and x
        return np.log(Z)
    
    def df(self, Z):    # Derivative of inverse nonlinearity
        return 1/(Z)
    
    def ddf(self, Z):   # Second derivative of inverse nonlinearity
        return -1/(Z)**2
    
    def F(self, z, u):    # Returns cost function value of given inputs z and u
        return np.linalg.norm(self.y - self.A@(z*u))**2 + self.lamb*np.linalg.norm(u)**2 + self.mu*np.linalg.norm(self.f(z))**2       
    
    def derivatives(self):    # Updates gradiant based upon current z and u
        self.grad =  2*(self.AtA@self.z - self.Aty) + 2*self.mu*np.diag(self.df(self.z).T[0])@self.f(self.z)
        if self.method == 'g':  # Gradient descent
            self.v = -self.grad  # Search direction
        if self.method == 'n': # Newton descent
            L, Q = np.linalg.eigh(2*self.AtA + 2*self.mu*np.diag((self.ddf(self.z)*self.f(self.z)+self.df(self.z)**2).T[0]))
            self.hessian = np.dot(Q*np.maximum(L,self.eps), Q.T) 
            self.v = np.linalg.solve(self.hessian, -self.grad) # Search direction
        
    # Performs an Armijo's backtracking line search for step sizes
    def backtrack(self):
        self.eta = 1      # Initial step size
        self.update = self.z + self.eta*self.v         # Calculate next z step
        while (min(self.update) < self.dom[0] or max(self.update) > self.dom[1]):
            self.eta = self.beta*self.eta
            self.update = self.z + self.eta*self.v
        cost = self.F(self.z, self.u)
        dd = self.tau*np.transpose(self.grad)@self.v
        f1 = self.F(self.update, self.u) 
        f2 = cost + self.eta*dd
        while f1 > f2:
            self.eta = self.beta*self.eta
            self.update = self.z + self.eta*self.v         # Calculate next z step
            f1 = self.F(self.update, self.u)
            f2 = cost + self.eta*dd
 
    def Projection(self, z):
        z = np.maximum(z, self.threshold[0])
        z = np.minimum(z, self.threshold[1])
        return z
    
    # Updates variables that depend on u and performs multiple Steepest Descent iterations to update z
    def est(self):    
        self.Au = self.A@np.diag(self.u.T[0])
        self.AtA = self.Au.T@self.Au
        self.Aty = self.Au.T@self.y
        for step in range(0, self.N):
            self.derivatives()
            if -np.transpose(self.grad)@self.v <= tol:  # -np.transpose(self.grad)@self.v = steepest descent norm (corresponding to step v) squared of gradient
                return step  # Return number of steps for convergence
            self.backtrack()
            self.z = self.update
        return -1  # Return -1 if steepest descent updates of z do not converge

# Class to set up and estimate u via Tikhonov solution
class Uest:
    
    # Class initilization. When calling Uest inputs for __init__ can be passed to it to create all of the variables below and initalize the class 
    def __init__(self, lamb, Psi, Phi, y, z):
        self.lamb = lamb  # Regularization parameter
        self.A = Psi@Phi   # Measurement Matrix
        self.z = z  # Initial z estimate
        self.y = y  # Measurements
    
    # Updates variables which depend on z and estimates u accordingly
    def est(self):
        self.Az = self.A@np.diag(self.z.T[0])
        self.AtA = self.Az.T@self.Az
        self.Aty = self.Az.T@self.y
        self.u = np.linalg.solve(self.AtA + self.lamb*np.eye(self.z.size), self.Aty)

# Testing on Barbara Snipit

In [ ]:
## Parameter Choices ##
mu, lamb = 2, 0.3  # Regularization parameters 'mu', 'lamb'.
tol = 10**(-6)           # Tolerance for Steepest Descent convergence
J = 1                  # Number of Steepest Descent steps for each estimation of z
K = 1000          # Number of times we alternate between estimating u and z 
method = 'g'    # Steepest descent method. Options: 'n' (Newton's Descent), 'g' (gradient descent)
if method == 'g':
    scale = 1  # Amount to scale measurement by
    threshold = (1,np.e**2)  # Thresholding range
    epsilon = None
elif method == 'n':
    scale = np.exp(-4)
    threshold = (1,np.e)
    epsilon = 10**(-5)
########################
y = scale*B.y
Psi = B.Psi
Phi = B.Phi

start = time.time()
Z = Zest(J, lamb, mu, Psi, Phi, y, method, tol, epsilon, threshold = threshold)        # Initialize Zest class
U = Uest(lamb, Psi, Phi, y, copy.deepcopy(Z.z))    # Initialize Uest class
U.est()  # Initial Estimate of u
k, pNorm, cNorm = 1, 0, Z.F(Z.z,U.u)
norms = [cNorm]  # Keep track of cost function values
iterate, norm, norm_df, z_steps = [], [], [], []
step = -1
while k <= K:
    if k % 1000 == 0:
        print('Iteration {}. Run time {:.2f}'.format(k, time.time()-start))
    Z.u = copy.deepcopy(U.u)  # Update u variable in Z class
    s = Z.est()   # Calculate J steepest descent update of z (or until convergence)
    U.z = copy.deepcopy(Z.z)   # Update z variable in U class
    U.est()   # Calculate Tikhonov update of u
    pNorm = cNorm
    cNorm = Z.F(Z.z, U.u) 
    norms.append(cNorm)
    if s != -1 and s != step:
        if s == 0:
            print('Converged in {} iterations with norm {:.4e}'.format(k,cNorm))
            iterate.append(k), norm.append(cNorm), norm_df.append(norms[-3]-cNorm), z_steps.append(s)
            break
        else:
            step = s
            print('Current iteration {} with norm {:.4e}. Num Newton steps {}'.format(k,cNorm,step))
            print('Difference in Cost Function Values = {:.4e}'.format(pNorm-cNorm))
            iterate.append(k), norm.append(cNorm), norm_df.append(pNorm-cNorm), z_steps.append(s)
    k += 1    

print('Run time = {}'.format(time.time()-start))
print('||y - A(z u))|| = ', np.linalg.norm(y - Z.A@(Z.z*U.u))**2)
print('\u03BC||f(z)|| = ', mu*np.linalg.norm(Z.f(Z.z))**2)
print('\u03BB||u|| = ', lamb*np.linalg.norm(U.u)**2, '\n')
I_est = np.reshape(np.matmul(Phi, Z.z*U.u), (img_size, img_size))
plt.imshow(I_est, cmap = 'gray')
print('MSE = {}'.format(np.linalg.norm(Z.z*U.u-scale*B.Phi.T@np.reshape(B.I, (img_size**2,1)))**2))
print('SSIM = {}'.format(ssim(scale*B.I,I_est,data_range = scale)))
plt.show()

In [ ]:
d = {'Iteration k': iterate, 'Cost Function F_k': norm, 'F_(k-1) - F_k': norm_df, 'Descent Steps': z_steps}
df = pd.DataFrame(data=d)
pd.set_option('display.float_format', '{:.4e}'.format)
df